In [5]:
!pip install --upgrade google-cloud-speech

/anaconda3/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.PersistentlyDeprecated2018,
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
Requirement already up-to-date: google-cloud-speech in /anaconda3/lib/python2.7/site-packages (1.2.0)
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
!export GOOGLE_APPLICATION_CREDENTIALS="/Users/anshul/Desktop/ISTEM/My First Project-ebb8cc0917d0.json"

In [7]:
def wer1(ref, hyp ,debug=False):
    ref=ref.lower()
    hyp=hyp.lower()
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]

    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    DEL_PENALTY=1 # Tact
    INS_PENALTY=1 # Tact
    SUB_PENALTY=1 # Tact
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL

    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1

                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL

    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append(".\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = list(reversed(lines))
        for i,line in enumerate(lines):
            if(line.startswith('.')):
                if(i-1 > 0):
                    if(not lines[i-1].startswith('.')):
                        print(line)
                        continue
                if(i+1 < len(lines)):
                    if(lines[i+1].startswith('.')):continue
            print(line)
        print("Ncor " + str(numCor))
        print("Nsub " + str(numSub))
        print("Ndel " + str(numDel))
        print("Nins " + str(numIns))
    return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return {'WER':wer_result, 'Cor':numCor, 'Sub':numSub, 'Ins':numIns, 'Del':numDel}

In [7]:
!pip3 install nltk
!pip3 install jiwer

    100% |████████████████████████████████| 1.5MB 500kB/s ta 0:00:012
  Stored in directory: /Users/anshul/Library/Caches/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
def lemmatized(para):
    para=para.lower()
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

        return tag_dict.get(tag, wordnet.NOUN)

    lemmatizer = WordNetLemmatizer()
    final=''
    sent_text = nltk.sent_tokenize(para) 
    for sentence in sent_text:
        x=([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])
        x=" ".join(x)+' '
        final=final+x
    return(final)

In [22]:
!pip3 install --upgrade google-cloud-speech
!pip3 install --upgrade google-cloud-bigquery

Requirement already up-to-date: google-cloud-speech in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (1.2.0)
You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 153kB 623kB/s ta 0:00:01


You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [23]:
import os
from google.cloud.bigquery.client import Client

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/anshul/Desktop/ISTEM/My First Project-ebb8cc0917d0.json"
bq_client = Client()

In [88]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums


def sample_long_running_recognize(audio):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()
    #client = speech.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.raw'

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 16000

    # The language of the supplied audio
    language_code = "en-US"

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.FLAC
    config = {
        "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "encoding": encoding,
    }
    #audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()
    #print(response.results)
    res = ''
    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
       
        #print(u"Transcript: {}".format(alternative.transcript))
        res += alternative.transcript
        
    return res
    


In [90]:
import io
import os
from pydub import AudioSegment
import soundfile as sf
# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
# Import the base64 encoding library.
import base64

# Pass the audio data to an encoding function.
def encode_audio(audio):
  audio_content = audio.read()
  return base64.b64encode(audio_content)
# Instantiates a client
client = speech.SpeechClient()
#enter path for audio files to be tested
path='/Users/anshul/Desktop/ISTEM/azure-speech-testing/'
#enter name of audio files to be tested
uris=[#'gs://speechtotext276/Kiran_Bedi_How_I_remade_one_of_Indias_toughest_prisonst0.flac',
      #'gs://speechtotext276/chem0.flac',
      #'gs://speechtotext276/chem1.flac',
      #'gs://speechtotext276/edu1.flac',
      'gs://speechtotext276/edu2.flac',
      'gs://speechtotext276/b0.flac',
      'gs://speechtotext276/b1.flac',
      'gs://speechtotext276/b2.flac',
      'gs://speechtotext276/Thoughts_on_humanity_fame_and_love_Shah_Rukh_Khant1.flac'
      
     ]
for uri in uris:
    audio = {"uri": uri}
    print(uri,'audio loaded')
    res = sample_long_running_recognize(audio)
    
    file = open('./auds/subs/'+uri[21:-5]+'.txt',"w")#write mode 
    file.write(res) 
    file.close()
    print('saved txt file')

audio loaded
Waiting for operation to complete...
audio loaded
Waiting for operation to complete...
audio loaded
Waiting for operation to complete...
audio loaded
Waiting for operation to complete...
audio loaded
Waiting for operation to complete...


In [87]:
file = open('./auds/subs/'+uri[21:-4]+'.txt',"w")#write mode 
file.write(res) 
file.close() 

In [61]:
transcripts = []
transcripts.append(res)

In [73]:
s

'0\tTranslator: Riaki Ponit Reviewer: Denise RQ I\'m here to talk to you about Indian education, higher education in particular. But I\'m actually going to start with demography. How many of you here are under 35? OK, that seems pretty representative of the country; 65% of India is under 35. How many of you are under 25? Then you are not representing because half of the Indian population is pretty much under 25. We are an amazingly young country. In fact, if you just take the age group from 10 to 19, there are 226 million Indians, poised, in other words, to enter higher education, going through school and ready for higher education. This is amazing because it\'s happening at the time when the rest of the world is aging. If you look at the average age in India today, it\'s 28. Of course, don\'t ask about the gap - since we heard about gaps - between the average age of the Indian person and of the Indian cabinet. I think we hold the world record for that. But, that\'s another TED talk, r

In [72]:
import os
ground_truths=[]
vids=['A_well_educated_mind_vs_a_well_formed_mind_Dr_Shashi_Tharoor_at_TEDxGateway_2013',
     ]
for file in vids:
    f= open((path+file+'_transcribed.txt'),'r')
    w=f.readlines()
    #print(w)
    ground_truths.append(w)
wers=[]
s = ''
for string in w:
    s += string
ground_truths = [s]
for i in range(len(transcripts)):
    ground=(ground_truths[i].split('\t')[1])
    print('\n',(ground),'\n') # printing the actual ground truth without any normalization
    print('\n',(transcripts[i]),'\n') # printing the actual transcripts without any normalization
    ground=lemmatized(ground)
    ground=" ".join(re.findall("[a-zA-Z0-9]+", ground)) # normalizing - removing punctuations
    ele=" ".join(re.findall("[a-zA-Z0-9]+",(lemmatized(transcripts[i])))) 
    wer1(ground,ele,True)#  comment this line if you only want the word error rate and not the debugging
    wers.append(wer(ground,ele))


 Translator: Riaki Ponit Reviewer: Denise RQ I'm here to talk to you about Indian education, higher education in particular. But I'm actually going to start with demography. How many of you here are under 35? OK, that seems pretty representative of the country; 65% of India is under 35. How many of you are under 25? Then you are not representing because half of the Indian population is pretty much under 25. We are an amazingly young country. In fact, if you just take the age group from 10 to 19, there are 226 million Indians, poised, in other words, to enter higher education, going through school and ready for higher education. This is amazing because it's happening at the time when the rest of the world is aging. If you look at the average age in India today, it's 28. Of course, don't ask about the gap - since we heard about gaps - between the average age of the Indian person and of the Indian cabinet. I think we hold the world record for that. But, that's another TED talk, right? Bu

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - '/Users/anshul/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.7/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.7/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.7/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
for i in range(len(vids)):
    print(vids[i],'\t',wers[i])